# Pandas merge and join

> A simple tutorial on pandas merge and joins
> using animation from https://github.com/gadenbuie/tidy-animated-verbs

# Setup

Let's create two DataFrames:

![x and y dataframes](tidy-animated-verbs/images/original-dfs.png)

In [1]:
import pandas as pd

In [2]:
x = pd.DataFrame(index=[1, 2, 3], data=[f'x{i}' for i in '123'], columns=['x_col'])
y = pd.DataFrame(index=[1, 2, 4], data=[f'y{i}' for i in '124'], columns=['y_col'])

In [3]:
x

,x_col
1,x1
2,x2
3,x3


In [4]:
y

,y_col
1,y1
2,y2
4,y4


## Inner join

The *intersection* of the keys:

![inner merge animation](tidy-animated-verbs/images/inner-join.gif)

In [5]:
x.join(y, how='inner')

,x_col,y_col
1,x1,y1
2,x2,y2


In [6]:
pd.merge(x, y, how='inner', left_index=True, right_index=True)

,x_col,y_col
1,x1,y1
2,x2,y2


# Mutating Joins

> *A mutating join allows you to combine variables (columns) from two tables. It
> first matches observations (rows) by their keys, then*
> [it adds all the variables (columns) from the input table in the output table].
>
> [R for Data Science: Mutating
> joins](http://r4ds.had.co.nz/relational-data.html#mutating-joins)


## Outer join

Also called **full join**, it performs the *union* of the keys:

![outer join animation](tidy-animated-verbs/images/full-join.gif)

In [7]:
x.join(y, how='outer')

,x_col,y_col
1,x1,y1
2,x2,y2
3,x3,NaN
4,NaN,y4


In [8]:
pd.merge(x, y, how='outer', left_index=True, right_index=True)

,x_col,y_col
1,x1,y1
2,x2,y2
3,x3,NaN
4,NaN,y4


## Left join

![left join animation](tidy-animated-verbs/images/left-join.gif)

In [9]:
x.join(y)

,x_col,y_col
1,x1,y1
2,x2,y2
3,x3,NaN


In [10]:
pd.merge(x, y, how='left', left_index=True, right_index=True)

,x_col,y_col
1,x1,y1
2,x2,y2
3,x3,NaN


## Right join

![right join animation](tidy-animated-verbs/images/right-join.gif)

In [11]:
y.join(x)

,y_col,x_col
1,y1,x1
2,y2,x2
4,y4,NaN


In [12]:
pd.merge(x, y, how='right', left_index=True, right_index=True)

,x_col,y_col
1,x1,y1
2,x2,y2
4,NaN,y4


## Left join with repeating right key

![left join with repeats animation](tidy-animated-verbs/images/left-join-extra.gif)

In [13]:
idx = pd.Index([1, 2, 4, 2], name='idx')
y2 = pd.DataFrame(index=idx, data=[f'y{i}' for i in idx], columns=['col'])
y2

,col
idx,
1,y1
2,y2
4,y4
2,y2


In [14]:
y2 = pd.DataFrame(index=[1, 2, 4, 2], data=[f'y{i}' for i in '1242'], columns=['col'])
y2

,col
1,y1
2,y2
4,y4
2,y2


In [15]:
x.join(y2)

,x_col,col
1,x1,y1
2,x2,y2
2,x2,y2
3,x3,NaN


In [16]:
pd.merge(x, y, how='left', left_index=True, right_index=True)

,x_col,y_col
1,x1,y1
2,x2,y2
3,x3,NaN


# Filtering Joins

> Filtering joins match observations (rows) in the same way as mutating joins,
> but affect the observations, not the variables (columns). … Semi-joins are
> useful for matching filtered summary tables back to the original rows.
> … Anti-joins are useful for diagnosing join mismatches.
>
> [R for Data Science: Filtering
> Joins](http://r4ds.had.co.nz/relational-data.html#filtering-joins)


## Semi Join

Take rows from the first DataFrame with keys present in the second:

![semi-join animation](tidy-animated-verbs/images/semi-join.gif)


In [17]:
rows = set(x.index).intersection(y.index)
x.loc[rows]

,x_col
1,x1
2,x2


## Anti-join

Take the rows of the first data frame with keys not present in the second:

![anti-join animation](tidy-animated-verbs/images/anti-join.gif)

In [18]:
rows = set(x.index).difference(y.index)
x.loc[rows]

,x_col
3,x3


# Set Operations

> All these operations work with a
> complete rows, comparing the values of every variable. These expect the
> x and y inputs to have the same variables *[columns]*, and treat 
> *[each row]* as *[an element of]* a set.  
> [R for Data Science: Set
> operations](http://r4ds.had.co.nz/relational-data.html#set-operations)

The two dataframes are treated as sets where an element is a row
and the index is ignored.
We can perform "set operations" on these two sets.

In [19]:
x = pd.DataFrame(data=[[1, 'a'], [1, 'b'], [2, 'a']])
y = pd.DataFrame(data=[[1, 'a'], [2, 'b'], [1, 'b']])

In [20]:
x

,0,1
0,1,a
1,1,b
2,2,a


In [21]:
y

,0,1
0,1,a
1,2,b
2,1,b


In [22]:
x_set = set(tuple(line) for line in x.values)
y_set = set(tuple(line) for line in y.values)

## Union / Intersection

In [23]:
pd.DataFrame(list(x_set.union(y_set)))

,0,1
0,2,a
1,1,a
2,2,b
3,1,b


In [24]:
pd.DataFrame(list(x_set.intersection(y_set)))

,0,1
0,1,a
1,1,b


## Using df.apply()

> It should be possible to use df.apply() for union intersection
> as in this SO question https://stackoverflow.com/questions/18180763/set-difference-for-pandas
>
>Need to work it out.

### Union with repeats

This is not strictly a set operation:

In [25]:
pd.concat([x, y])

,0,1
0,1,a
1,1,b
2,2,a
0,1,a
1,2,b
2,1,b


Or, ignoring the index:

In [26]:
pd.concat([x, y], ignore_index=True)

,0,1
0,1,a
1,1,b
2,2,a
3,1,a
4,2,b
5,1,b


# Tidy Data

[Tidy data](http://r4ds.had.co.nz/tidy-data.html#tidy-data-1) (Hadley Wickham 2013) follows the following three rules:

1. Each variable has its own column.
2. Each observation has its own row.
3. Each value has its own cell.

![wide long dataframes](tidy-animated-verbs/images/static/png/original-dfs-tidy.png)

## Melt / Pivot

*or Spread / Gather*

![spread animation](tidy-animated-verbs/images/tidyr-spread-gather.gif)

**Melt**

- Output has two columns "variable" and "value"
- Each cell values goes to the "value" column
- The column name of each value, goes in the "variable" column


In [27]:
data = {'x': ['a', 'b'], 'y': ['c', 'd'], 'z': ['e', 'f'],}
df_wide = (pd.DataFrame(data)
           .rename_axis('id', axis='index')
           .rename_axis('key', axis='columns'))

In [28]:
df_wide

key,x,y,z
id,,,
0,a,c,e
1,b,d,f


In [29]:
df_tidy = (df_wide
           .reset_index()
           .melt(id_vars='id')
           .set_index('id'))
df_tidy

,key,value
id,,
0,x,a
1,x,b
0,y,c
1,y,d
0,z,e
1,z,f


In [30]:
df_tidy.pivot(columns='key', values='value')

key,x,y,z
id,,,
0,a,c,e
1,b,d,f


In [31]:
df_tidy.pivot(columns='key', values='value') == df_wide

key,x,y,z
id,,,
0,True,True,True
1,True,True,True


# Wide vs Tidy

## Wide

- More suitable for **gridded** data
- Compact representation (gridded case)
- Always possible to transform to **tidy**

## Tidy

- More general format
- Good abstraction for wide range of data
- Not always possible/meaningful to transform to **wide**

## Melt: change names of axis

Change the name of the "variable" / "value" columns:

In [32]:
df_wide.melt(var_name='variables', value_name='values')

,variables,values
0,x,a
1,x,b
2,y,c
3,y,d
4,z,e
5,z,f


# Lossy Melt

*melt while dropping columns*

Melt (or "spread") the DF using **selected columns** name as "variable" and each column element as a "value". Unselected columns are dropped.

In [33]:
df_wide.melt(value_vars=['x', 'y'])

,key,value
0,x,a
1,x,b
2,y,c
3,y,d


Melt (or "spread") the DF using **selected columns** name as "index" column. The remaining columns are melted in two "variable" / "value" columns. 

In [34]:
df_wide.melt(id_vars=['x'])

,x,key,value
0,a,y,c
1,b,y,d
2,a,z,e
3,b,z,f


Note that melting only the non-id columns produce the same 
"variable" / "value" columns but without the "id" column 'x':

In [35]:
pd.melt(df_wide, value_vars=['y', 'z'])

,key,value
0,y,c
1,y,d
2,z,e
3,z,f


In [36]:
df = pd.DataFrame({'A': {0: 'a', 1: 'b', 2: 'c'},
                   'B': {0: 1, 1: 3, 2: 5},
                   'C': {0: 2, 1: 4, 2: 6}})
df

,A,B,C
0,a,1,2
1,b,3,4
2,c,5,6


In [37]:
df.melt()

,variable,value
0,A,a
1,A,b
2,A,c
3,B,1
4,B,3
5,B,5
6,C,2
7,C,4
8,C,6
